In [49]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [50]:
#load data
housing_df = pd.read_csv('HousingData.csv')
housing_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


In [51]:
#drop null values
housing_df = housing_df.dropna()

In [52]:
#declare X and y
#X is predictor columns, so we want everything but the last column
#y is what we're trying to predict, the value, so we just need the last column
X = housing_df.iloc[:,:-1]
y = housing_df.iloc[:, -1]

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,
                                                    random_state=0)

In [54]:
#create empty linear regression model
reg = LinearRegression()

#and fit the data to the model
reg.fit(X_train, y_train)

LinearRegression()

In [55]:
#now that we have the training model, test the accuracy
#predict on test data: y_pred
y_pred = reg.predict(X_test)

In [56]:
#compute and print RMSE (the error in the prediction)
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print(f'RMSE: {rmse}')

RMSE: 5.371207757773577


In [57]:
#since the median value is in the thousands, this means we have an error of about $5,370

In [58]:
def regression_model(model):
    #create training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
    #create regressor: reg_all
    reg_all = model
    #fit regressor to training data
    reg_all.fit(X_train, y_train)
    #predict on the test data: y_pred
    y_pred = reg_all.predict(X_test)
    #compute error RMSE
    rmse = np.sqrt(mean_squared_error(y_test,y_pred))
    print(f'RMSE: {rmse}')
    

In [59]:
#now we have it all as a function, we can use it multiple times on the model we created.
regression_model(LinearRegression())

RMSE: 3.60170880119608


In [60]:
regression_model(LinearRegression())

RMSE: 4.009970741605161


In [61]:
#too much variance because of the variability of the training sets. ONWARD!

Next we have to do cross validation.  The training data will be split into folds, (usually 3, 5, or 10) 5 is standard.  The algorithm is fit on one fold at a time, and tested on the remaining data.   This results in 5 different training sets, and test sets, the mean of the scores is considered the accuracy of the model.

In [62]:
#cross_val_score does all of that for us so let's import it.
from sklearn.model_selection import cross_val_score

In [63]:
def regression_model_cv(model, k=5):
    scores = cross_val_score(model, X, y,
                             scoring='neg_mean_squared_error', cv=k)
    rmse = np.sqrt(-scores)
    print('Reg rmse:', rmse)
    print('Reg mean:', rmse.mean())

In [64]:
regression_model_cv(LinearRegression())

Reg rmse: [3.26123843 4.42712448 5.66151114 8.09493087 5.24453989]
Reg mean: 5.3378689628783516


In [65]:
#trying with 3 folds
regression_model_cv(LinearRegression(),k=3)

Reg rmse: [ 3.72504914  6.01655701 23.20863933]
Reg mean: 10.983415161090788


In [66]:
#trying with 6 folds
regression_model_cv(LinearRegression(),k=6)

Reg rmse: [3.23879491 3.97041949 5.58329663 3.92861033 9.88399671 3.91442679]
Reg mean: 5.086590810801092


There's a lot of variance here, and that can be due to a small data set, or because of outliers, and that'd have to be fixed with some regularization.


Ridge and Lasso are common methods to fix overfitting when regularizing by adding penalties to higher order polynomials (additional columns and their weights).  Ridge is used more often.

In [67]:
from sklearn.linear_model import Ridge
regression_model_cv(Ridge())

Reg rmse: [3.17202127 4.54972372 5.36604368 8.03715216 5.03988501]
Reg mean: 5.23296516625177


In [68]:
from sklearn.linear_model import Lasso
regression_model_cv(Lasso())

Reg rmse: [3.52318747 5.70083491 7.82318757 6.9878025  3.97229348]
Reg mean: 5.60146118538429


Trying different methods to see what works best is usually a good idea when trying to avoid overfitting or underfitting the data.

In [69]:
#trying the k-nearest neighbors regressor which compares to other data points nearby
from sklearn.neighbors import KNeighborsRegressor
regression_model_cv(KNeighborsRegressor())

Reg rmse: [ 8.24568226  8.81322798 10.58043836  8.85643441  5.98100069]
Reg mean: 8.495356738515685


In [70]:
#that was with 5 neighbors, lets try with other amounts
regression_model_cv(KNeighborsRegressor(n_neighbors=4))

Reg rmse: [ 8.44659788  8.99814547 10.97170231  8.86647969  5.72114135]
Reg mean: 8.600813339223432


In [71]:
regression_model_cv(KNeighborsRegressor(n_neighbors=7))

Reg rmse: [ 7.99710601  8.68309183 10.66332898  8.90261573  5.51032355]
Reg mean: 8.351293217401393


In [72]:
regression_model_cv(KNeighborsRegressor(n_neighbors=10))

Reg rmse: [ 7.47549287  8.62914556 10.69543822  8.91330686  6.52982222]
Reg mean: 8.448641147609868


In [73]:
#add GridSearchCV to check all possible values in a grid to find the best number of neighbors
from sklearn.model_selection import GridSearchCV


In [74]:
#check between 1 and 20 neighbors.  params are first num to check, second is last one, then interval
neighbors = np.linspace(1, 20, 20)

In [75]:
#convert to int. required for knn
k = neighbors.astype(int)

In [76]:
#the grid goes into a dict
param_grid = {'n_neighbors': k}

In [77]:
#build the model for each neighbor
knn = KNeighborsRegressor()

In [78]:
#instantiate the grid search
knn_tuned = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')

In [79]:
#fit it to the data
knn_tuned.fit(X,y)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])},
             scoring='neg_mean_squared_error')

In [80]:
#and print the results for the best params
k=knn_tuned.best_params_
print(f'Best n_neighbors: {k}')
score = knn_tuned.best_score_
rsm = np.sqrt(-score)
print(f'Best score: {rsm}')

Best n_neighbors: {'n_neighbors': 7}
Best score: 8.516767055977628


Now we try decision trees, which is the equivalent of 20 questions where you ask yes/no, to narrow down toward an answer.   Prone to overfitting, but useful in creating random forests


Then Random Forest which work with both regression and classification and often performs well. It is an ensemble of decision trees.  It may consist of hundreds of decision trees.  It is usually able to generalize much better.

In [81]:
from sklearn import tree
regression_model_cv(tree.DecisionTreeRegressor(random_state=0))

Reg rmse: [3.7647936  7.26184759 7.78346186 6.48142428 4.79234165]
Reg mean: 6.016773796161434


In [82]:
from sklearn.ensemble import RandomForestRegressor
regression_model_cv(RandomForestRegressor(random_state=0))

Reg rmse: [3.21859405 3.76199072 4.96431026 6.55950671 3.7700697 ]
Reg mean: 4.454894289804201


Best result so far, so let's explore RandomForest hyperparameters

n_jobs defaults to none and it relates to the number of processing jobs that can be run.  -1 means use all available processors

n_estimators defaults to 100 and it's the number of trees to use in the ensemble, usually good to start with 100, then maybe go to 500, going big uses way more ram and may not make that much of a difference.

max_depth (def is none) max depth of the trees. none means no limit
min_samples_split defs to 2 and means the minimum number of samples that has to exist for a split to happen.
min_samples_leaf is def to 1 and means it's the min num of smples as the leaves/base of tree 
max_features def to "auto" this is the num of features to consider when looking for a good split.  for classification sqrt is recommended.

GridSearchCV can be used here to find the best params but that could be millions of combinations.
RandomizedSearchCV comes up with 10 random combinations of params.


In [83]:
from sklearn.model_selection import RandomizedSearchCV

In [84]:
param_grid = {'max_depth': [None, 1, 2, 3, 4, 5, 6, 8, 10, 15, 20],
              'min_samples_split': [2, 3, 4, 5, 6],
              'min_samples_leaf': [1, 2, 3, 4, 6, 8],
              'max_features': [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4]}

In [85]:
reg = RandomForestRegressor(n_jobs=-1, random_state=0)

In [86]:
reg_tuned = RandomizedSearchCV(reg, param_grid, cv=5, 
                               scoring='neg_mean_squared_error',
                               random_state=0)

In [87]:
reg_tuned.fit(X,y)

RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=0),
                   param_distributions={'max_depth': [None, 1, 2, 3, 4, 5, 6, 8,
                                                      10, 15, 20],
                                        'max_features': [1.0, 0.9, 0.8, 0.7,
                                                         0.6, 0.5, 0.4],
                                        'min_samples_leaf': [1, 2, 3, 4, 6, 8],
                                        'min_samples_split': [2, 3, 4, 5, 6]},
                   random_state=0, scoring='neg_mean_squared_error')

In [88]:
p=reg_tuned.best_params_
print(f'Best params: {p}')
score = reg_tuned.best_score_
rsm = np.sqrt(-score)
print(f'Best score: {rsm}')

Best params: {'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 0.7, 'max_depth': 10}
Best score: 4.46557417781969


and even with all of that, it did not perform as well as the defaults, but now lets try with 500 tree ensemble


In [89]:
regression_model_cv(RandomForestRegressor(n_jobs=-1, n_estimators=500))

Reg rmse: [3.24224625 3.68466219 4.7243451  6.55938849 3.94955915]
Reg mean: 4.4320402390414575


it did better, so now lets try it with a 500 tree ensemble AND the tuned params it came up with.

In [90]:
regression_model_cv(RandomForestRegressor(n_jobs=-1,
                                          n_estimators=500,
                                          random_state=0,
                                          min_samples_split=5,
                                          min_samples_leaf=2,
                                          max_features=0.7,
                                          max_depth=10))

Reg rmse: [3.18498898 3.59234342 4.66618434 6.43013587 3.81099639]
Reg mean: 4.336929799775126


best score yet!  tuned params help prevent overfitting, and larger ensembles usually have less error, but also take longer to build the model.

In [91]:
from sklearn.ensemble import AdaBoostRegressor
regression_model_cv(AdaBoostRegressor())

Reg rmse: [3.58763943 3.35372816 5.4529453  6.41137795 4.06069505]
Reg mean: 4.573277177220059


In [92]:
from xgboost import XGBRegressor
regression_model_cv(XGBRegressor())

Reg rmse: [3.11376639 5.36460979 6.15243328 6.64486279 4.12696925]
Reg mean: 5.080528301575429
